In [1]:
import tensorflow as tf
from tensorflow import keras

In [4]:
! pip install -q -U keras-tuner 

In [7]:
import keras_tuner as kt

In [8]:
(img_train,label_train),(img_test,label_test)= keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 1s 0us/step


In [9]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [15]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    
    hp_units = hp.Int('units',min_value= 28, max_value=40,step=2)
    
    model.add(keras.layers.Dense(units=hp_units,activation='relu'))
    model.add(keras.layers.Dense(10))
    
    hp_learning = hp.Choice('learning rate',values=[0.001,0.0001,0.00001])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning),
                                                 loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                                                 metrics=['accuracy'])
    return model

In [16]:
tuner = kt.Hyperband(model_builder,objective='val_accuracy',
                    max_epochs=2,
                    directory='my_dir',
                    project_name='intro_to_kt',
                    overwrite=True)

In [17]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             patience=5)

In [18]:
tuner.search(img_train,label_train,
            epochs=2, validation_split=0.2,
            callbacks=[stop_early])

Trial 2 Complete [00h 00m 02s]
val_accuracy: 0.8580833077430725

Best val_accuracy So Far: 0.8580833077430725
Total elapsed time: 00h 00m 05s
INFO:tensorflow:Oracle triggered exit


In [22]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [23]:
best_hps.get('units')


36

In [25]:
best_hps.get('learning rate')

0.001

In [26]:
model = tuner.hypermodel.build(best_hps)

In [30]:
history = model.fit(img_train,label_train,
                   epochs=10,validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 1s 691us/step - loss: 0.3312 - accuracy: 0.8805 - val_loss: 0.3736 - val_accuracy: 0.8684
Epoch 2/10
1500/1500 [==============================] - 1s 714us/step - loss: 0.3214 - accuracy: 0.8834 - val_loss: 0.3476 - val_accuracy: 0.8788
Epoch 3/10
1500/1500 [==============================] - 1s 700us/step - loss: 0.3097 - accuracy: 0.8878 - val_loss: 0.3785 - val_accuracy: 0.8633
Epoch 4/10
1500/1500 [==============================] - 1s 732us/step - loss: 0.3009 - accuracy: 0.8902 - val_loss: 0.3569 - val_accuracy: 0.8728
Epoch 5/10
1500/1500 [==============================] - 1s 941us/step - loss: 0.2940 - accuracy: 0.8923 - val_loss: 0.3582 - val_accuracy: 0.8712
Epoch 6/10
1500/1500 [==============================] - 1s 709us/step - loss: 0.2864 - accuracy: 0.8951 - val_loss: 0.3405 - val_accuracy: 0.8777
Epoch 7/10
1500/1500 [==============================] - 1s 810us/step - loss: 0.2792 - accuracy: 0.8968 - val_loss: 0.3575 -

In [31]:
#to get the best epoch

valacc = history.history['val_accuracy']
best_epoch = valacc.index(max(valacc))+1
best_epoch

10

In [32]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(img_train,label_train,epochs= best_epoch,validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.5670 - accuracy: 0.8042 - val_loss: 0.5025 - val_accuracy: 0.8197
Epoch 2/10
1500/1500 [==============================] - 1s 735us/step - loss: 0.4188 - accuracy: 0.8529 - val_loss: 0.4259 - val_accuracy: 0.8524
Epoch 3/10
1500/1500 [==============================] - 1s 798us/step - loss: 0.3847 - accuracy: 0.8643 - val_loss: 0.3829 - val_accuracy: 0.8631
Epoch 4/10
1500/1500 [==============================] - 1s 777us/step - loss: 0.3599 - accuracy: 0.8710 - val_loss: 0.4044 - val_accuracy: 0.8631
Epoch 5/10
1500/1500 [==============================] - 1s 972us/step - loss: 0.3442 - accuracy: 0.8753 - val_loss: 0.3613 - val_accuracy: 0.8742
Epoch 6/10
1500/1500 [==============================] - 1s 822us/step - loss: 0.3294 - accuracy: 0.8806 - val_loss: 0.3689 - val_accuracy: 0.8698
Epoch 7/10
1500/1500 [==============================] - 1s 801us/step - loss: 0.3208 - accuracy: 0.8833 - val_loss: 0.3780 - v

In [33]:
hypermodel.evaluate(img_test,label_test)

313/313 [==============================] - 0s 744us/step - loss: 0.3748 - accuracy: 0.8693


[0.3747963309288025, 0.8693000078201294]